# Import the material prices into S/4HANA

To view the material sales prices, use Transaction `VK33` > Prices > Material Price, Condition Type = `PR00`.  
Note that the currency depends on the Sales Org. (VKORG).

How can we change the prices (in Global Bike, we have only one price constant over time)?  
Originally, we intended to do this in ABAP. There is `BAPI_PRICES_CONDITIONS`. But according to Note 1135984, the BAPI must not be used and there is no other API available except using a modern OData service. 

The relevant OData V2 service is "Condition Record for Pricing in Sales", see documentation <https://api.sap.com/api/OP_API_SLSPRCGCONDITIONRECORD_SRV_0001/overview>.

## Configure OData Service in S4

To configure the service (cross-client customizing should be allowed):
- Transaction `/n/iwfnd/maint_service`
- Button "Add Service": Filter for System Alias = LOCAL, External Service Name = API_SLSPRICINGCONDITIONRECORD_SRV. 
- Button "Add Selected Services"

To test a price update in SAP Gateway Client:  
Request URI `https://<host>/sap/opu/odata/SAP/API_SLSPRICINGCONDITIONRECORD_SRV/A_SlsPrcgConditionRecord`  
and payload
``` json
{
  "ConditionTable" : "304",
  "ConditionType" : "PR00",
  "ConditionRateValue" : "200",
  "ConditionRateValueUnit" : "EUR",
  "to_SlsPrcgCndnRecdValidity" : [
    {
      "ConditionValidityStartDate" : "1990-01-01T00:00:00",
      "ConditionValidityEndDate" : "1990-12-31T00:00:00",
      "SalesOrganization" : "DN00",
      "DistributionChannel" : "WH",
      "Material" : "DXTR1000"
    }
  ]
}
```
Tested successfully: no matter how the date range of [ConditionValidityEndDate, ConditionValidityStartDate] is set (potentially overlapping to existing ranges), the price conditions are maintained correctly.

This is an example to do a test of the service in a web browser using a GET (just handy, not relevant)
`https://<host>/sap/opu/odata/sap/API_SLSPRICINGCONDITIONRECORD_SRV/A_SlsPrcgCndnRecdValidity/?$filter=(ConditionType%20eq%20%27PR00%27%20and%20Material%20eq%20%27DXTR1000%27)&$orderby=SalesOrganization%20asc&$select=*&$inlinecount=allpages`

## SAP-User for the import

We created a user `ANALYT_ODATA` with role `ZUCC_ANALYTICS_ODATA`.
The role is downloaded to file `ZUCC_ANALYTICS_ODATA.SAP` here.
Privileges are:
| Auth object | Field | Value |  |
|-------------|-------|-------|--|
| S_SERVICE   | SRV_NAME | IWSG / ZAPI_SLSPRICINGCONDITIONRECORD_SRV_0001 | Access OData service (not working ;-()
| S_SERVICE   | SRV_NAME | * | Since restricting to the above technical service name is not working, we give full access instead
|             | SRV_TYPE | HT
| V_KONH_VKS  | KSCHL    | PR00 | Create price condition
|             | ACTVT    | *
| B_BUP_PCPT  | ACTVT    | 3   | Display Business Partner
| V_KONH_VKO  | VKORG    | *   | Create condition for sales org
|             | VTWEG    | *
|             | SPART    | *
|             | ACTVT    | *

## Do the import via OData
### Read intended prices from `masterdata.xlsx`

In [1]:
import pandas as pd
%xmode minimal

Exception reporting mode: Minimal


Read master data

In [2]:
masterdata = '../generator/masterdata.xlsx'
prices_eur = pd.read_excel(masterdata, sheet_name="prices EUR")
prices_usd = pd.read_excel(masterdata, sheet_name="prices USD")
prices_eur.sample()

,MATNR,MAKTX,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
9,FAID1000,First Aid Kit,40,40.4,42.4,46.2,48.0,49.9,51.9,54.0,56.2,58.4,60.7,63.1,65.6,68.2,70.9,73.7


Convert prices to tidy format and add sales organizations

In [3]:
prices = (
    # combine both tables
    pd.concat([prices_eur, prices_usd], keys=["EUR", "USD"], names=["Currency"])
    .reset_index("Currency")
    # material description not needed
    .drop(columns="MAKTX")
    # tidy
    .melt(id_vars=["MATNR", "Currency"], var_name="Year", value_name="Price")
    # for each currency we have two sales organizations
    .assign(vkorg1=lambda df: df["Currency"].map({'EUR':'DN00', 'USD':'UW00'}))
    .assign(vkorg2=lambda df: df["Currency"].map({'EUR':'DS00', 'USD':'UE00'}))
    .melt(id_vars=["MATNR", "Currency", "Year", "Price"], value_name="VKORG")
    .drop(columns="variable")
    # only aesthetics
    .reindex(columns=["VKORG", "MATNR", "Year", "Price", "Currency"])
)
prices.sample()

,VKORG,MATNR,Year,Price,Currency
725,DN00,DXTR1000,2035,5551.8,EUR


### Import prices to S4

`pyodata` is not part of anaconda. To install: `pip install -U pyodata`.  
Version >= 1.10.1.  
Documentation: <https://pyodata.readthedocs.io/en/latest/usage/advanced.html>

In [4]:
import requests
import pyodata
import Credentials # local modules must start with capital letter
from importlib import reload
reload(Credentials) # just in case; be able to reload here without restarting the kernel

<module 'Credentials' from 'c:\\Users\\verbarg\\Nextcloud\\Forschung\\2022 Reporting UCC\\2022-12 GeneratorGB\\gb-salesdata\\abap\\Credentials.py'>

In [5]:
# To get more logging from pyodata use logging.DEBUG
import logging

logging.basicConfig()
root_logger = logging.getLogger()
root_logger.setLevel(logging.ERROR)

Connect to OData service

In [6]:
service_url = f'{Credentials.p_host}:443/sap/opu/odata/sap/API_SLSPRICINGCONDITIONRECORD_SRV/'
session = requests.Session()
session.auth = (Credentials.p_username, Credentials.p_password)
session.params = {'sap-client': Credentials.p_client}   # not necessary, client is already defined in the service
# CSRF Token
response = session.head(service_url, headers={'x-csrf-token': 'fetch'})
token = response.headers.get('x-csrf-token', '')
session.headers.update({'x-csrf-token': token})
# service
prices_service = pyodata.Client(service_url, session)
prices_request = prices_service.entity_sets.A_SlsPrcgConditionRecord.create_entity()

payload

In [7]:
def price_condition(s):
  return {
    "ConditionTable" : "304",   # Condition table 304 (Material with release status)
    "ConditionType" : "PR00",   # net price in Global Bike
    "ConditionRateValue" : str(s.Price),
    "ConditionRateValueUnit" : s.Currency,
    "to_SlsPrcgCndnRecdValidity" : [
      {
        "ConditionValidityStartDate" : f'{s.Year}-01-01T00:00:00',
        "ConditionValidityEndDate" : f'{s.Year}-12-31T00:00:00',
        "SalesOrganization" : s.VKORG,
        "DistributionChannel" : "WH",
        "Material" : s.MATNR
      }
    ]
  }

For all price conditions...

In [8]:
for s in prices.itertuples():
    prices_request.set(**price_condition(s))  # **kwargs expandiert das dictionary
    try:
        prices_request.execute()  # do the import into S4
    except pyodata.exceptions.HttpError as ex:
        print(ex.response.text)
        print(f'{s.MATNR} {s.Year}')